In [1]:
import torch
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print(device)

!nvidia-smi

cuda
Tue Dec  1 01:27:50 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    29W / 300W |     10MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+--

In [2]:
!git clone https://github.com/nsusanj/temp.git

Cloning into 'temp'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 24 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (24/24), done.


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
!git clone https://github.com/adapter-hub/adapter-transformers.git
%cd adapter-transformers

Cloning into 'adapter-transformers'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 42519 (delta 10), reused 8 (delta 2), pack-reused 42490
Receiving objects: 100% (42519/42519), 23.88 MiB | 4.89 MiB/s, done.
Resolving deltas: 100% (30833/30833), done.
/content/adapter-transformers


In [5]:
!pip install .
!pip install sklearn

Processing /content/adapter-transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.9MB 11.9MB/s 
     |████████████████████████████████| 890kB 56.9MB/s 
     |████████████████████████████████| 1.1MB 54.2MB/s 
  Created wheel for adapter-transformers: filename=adapter_transformers-1.1.0-cp36-none-any.whl size=1325191 sha256=634fd0184b652f4dcfc201550ba28dd0f26a5cc31b150d3a15e64773a645b4a6
  Stored in directory: /root/.cache/pip/wheels/69/f2/bd/f0720d145cbd02c42f9d833e06bb319e9ee518b24ebaa092e1
Successfully built adapter-transformers
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=0c6de84d5d3f7046a97e0f1deb60cfe8b58a2336cbdad697758d6abe345e23d2
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [6]:
%cd ..

/content


In [8]:
%cd temp

/content/temp


In [9]:
!ls

adapticons	    Makefile   poetry.lock     README.md
docker-compose.yml  notebooks  pyproject.toml  setup.cfg


In [10]:
%cd adapticons

/content/temp/adapticons


In [12]:
%mkdir datasets

In [13]:
%cd datasets/
%mkdir chemprot
%cd chemprot

/content/temp/adapticons/datasets
/content/temp/adapticons/datasets/chemprot


In [14]:
!curl -Lo train.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/chemprot/train.jsonl
!curl -Lo dev.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/chemprot/dev.jsonl
!curl -Lo test.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/chemprot/test.jsonl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1094k  100 1094k    0     0   642k      0  0:00:01  0:00:01 --:--:--  642k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  662k  100  662k    0     0   435k      0  0:00:01  0:00:01 --:--:--  435k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  937k  100  937k    0     0   535k      0  0:00:01  0:00:01 --:--:--  535k


In [15]:
%cd ../..

/content/temp/adapticons


In [27]:
%cd modeling

/content/temp/adapticons/modeling


In [31]:
!ls ../../../gdrive/MyDrive/roberta_tapt_chem_01/

checkpoint-1000  config.json	    special_tokens_map.json  vocab.json
checkpoint-1500  merges.txt	    tokenizer_config.json
checkpoint-500	 pytorch_model.bin  training_args.bin


In [37]:
!python new_train.py \
  --do_train \
  --do_eval \
  --data_dir ../datasets/chemprot/ \
  --max_seq_length 512 \
  --per_device_train_batch_size 16 \
  --gradient_accumulation_steps 1 \
  --learning_rate 2e-5 \
  --num_train_epochs 10 \
  --output_dir results/baseline/chemprot/ \
  --task_name chemprot \
  --do_predict \
  --load_best_model_at_end \
  --model_name_or_path ../../../gdrive/MyDrive/roberta_tapt_chem_01/ \
  --metric micro \

2020-12-01 01:50:34.801321: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
PyTorch: setting up devices
12/01/2020 01:50:36 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
12/01/2020 01:50:36 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='results/baseline/chemprot/', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=True, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=16, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=2e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=10.0, max_steps=-1, warmup_steps=0, logging_dir='runs/

In [42]:
!ls results/baseline/chemprot

config.json		   pytorch_model.bin	      tokenizer_config.json
eval_results_chemprot.txt  special_tokens_map.json    training_args.bin
merges.txt		   test_results_chemprot.txt  vocab.json


In [43]:
!cp -r results/baseline/chemprot ../../../gdrive/MyDrive/

In [44]:
!ls ../../../gdrive/MyDrive/

 2010s-Books-Read.txt
'2015 Fantasy Baseball Draft.gsheet'
 2017_1010-Demystifying-NNs.pptx
 2019_TaxReturn.pdf.pdf
 21.s.susanj.gdoc
 22.gdoc
 28th:.gdoc
 4.s.football.gdoc
'6:20 pm - June 13: *Waiting, realizin....gdoc'
 6710-Funderburg-IMG_6338.jpg
 6710-Funderburg-IMG_6339.jpg
'Agile for Research Science.gdoc'
'Another Letter to No One.gdoc'
'Band Names.gdoc'
'Baseball Player Phone Numbers.gdoc'
'Ben Folds and the Big Blog News.gdoc'
 Bits.gdoc
'Blog Notes.gdoc'
'Buy-vs-Rent-Calculator-developer-version-C (1).gsheet'
'Buy-vs-Rent-Calculator-developer-version-C (1).xlsx'
'Buy-vs-Rent-Calculator-developer-version-C (2).gsheet'
'Buy-vs-Rent-Calculator-developer-version-C (3).gsheet'
'Buy-vs-Rent-Calculator-developer-version-C (4).gsheet'
 Buy-vs-Rent-Calculator-developer-version-C.gsheet
 Buy-vs-Rent-Calculator-developer-version-C-PIT.gsheet
 Buy-vs-Rent-Calculator-developer-version-C.xlsx
'Cell Phone Plans.gsheet'
 chemprot
'Christmas 2012 Gift Ideas.gdoc'
 Clothes.gdoc
'Colab Notebo

In [45]:
!ls

download_glue.py      results
glue_training.py      run_language_modeling_with_adapter_fusion.py
jsonl_to_txt.py       run_language_modeling_with_adapters.py
load_fusion_model.py  runs
new_train.py	      sample_pct_txt_file.py


In [50]:
%cd /content

/content


In [51]:
!ls

adapter-transformers  gdrive  sample_data  temp


In [52]:
%cd /content/temp/adapticons/datasets

/content/temp/adapticons/datasets


In [53]:
%mkdir citation

In [54]:
%cd citation

/content/temp/adapticons/datasets/citation


In [56]:
%cd /content

/content


In [57]:
!ls

adapter-transformers  gdrive  sample_data  temp


In [59]:
!pip install git+https://github.com/kernelmachine/allennlp.git@4ae123d2c3bfb1ea3ce7362cb6c5bca3d094ffa7
!pip install transformers==2.6.0
!pip install pytorch-transformers==1.2.0
!git clone https://github.com/allenai/dont-stop-pretraining

  Cloning https://github.com/kernelmachine/allennlp.git (to revision 4ae123d2c3bfb1ea3ce7362cb6c5bca3d094ffa7) to /tmp/pip-req-build-7jtr5kgy
  Running command git clone -q https://github.com/kernelmachine/allennlp.git /tmp/pip-req-build-7jtr5kgy
  Running command git checkout -q 4ae123d2c3bfb1ea3ce7362cb6c5bca3d094ffa7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 266kB 11.6MB/s 
     |████████████████████████████████| 30.9MB 186kB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 245kB 56.8MB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
     |████████████████████████████████| 71kB 9.5MB/s 
     |████████████████████████████████| 317kB 60.6MB/s 
     |████████████████████████████████| 133kB 59.3MB/s 
     |████████████████████████████████| 133kB 67.0MB/s 
     |████████████████████████████████| 163kB 63.2MB/s

In [60]:
%cd /content/dont-stop-pretraining

/content/dont-stop-pretraining


In [62]:
!curl -Lo cittrain.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/citation_intent/train.jsonl
!curl -Lo citdev.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/citation_intent/dev.jsonl
!curl -Lo citest.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/citation_intent/test.jsonl

!curl -Lo hyptrain.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/hyperpartisan_news/train.jsonl
!curl -Lo hypdev.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/hyperpartisan_news/dev.jsonl
!curl -Lo hyptest.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/hyperpartisan_news/test.jsonl

!curl -Lo sciietrain.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/sciie/train.jsonl
!curl -Lo sciiedev.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/sciie/dev.jsonl
!curl -Lo sciieest.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/sciie/test.jsonl

import pandas as pd
import json
data = []
with open("hyptrain.jsonl", 'r', encoding='utf-8') as f:
    for line in f:
       data.append(json.loads(line))
df = pd.DataFrame(data)
df.text.to_csv("hyptrain.txt", header=False, index=False)
data = []
with open("hypdev.jsonl", 'r', encoding='utf-8') as f:
    for line in f:
       data.append(json.loads(line))
df = pd.DataFrame(data)
df.text.to_csv("hypdev.txt", header=False, index=False)


data = []
with open("cittrain.jsonl", 'r', encoding='utf-8') as f:
    for line in f:
       data.append(json.loads(line))
df = pd.DataFrame(data)
df.text.to_csv("cittrain.txt", header=False, index=False)
data = []
with open("citdev.jsonl", 'r', encoding='utf-8') as f:
    for line in f:
       data.append(json.loads(line))
df = pd.DataFrame(data)
df.text.to_csv("citdev.txt", header=False, index=False)

data = []
with open("sciietrain.jsonl", 'r', encoding='utf-8') as f:
    for line in f:
       data.append(json.loads(line))
df = pd.DataFrame(data)
df.text.to_csv("sciietrain.txt", header=False, index=False)
data = []
with open("sciiedev.jsonl", 'r', encoding='utf-8') as f:
    for line in f:
       data.append(json.loads(line))
df = pd.DataFrame(data)
df.text.to_csv("sciiedev.txt", header=False, index=False)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  454k  100  454k    0     0   325k      0  0:00:01  0:00:01 --:--:--  325k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 30138  100 30138    0     0  35084      0 --:--:-- --:--:-- --:--:-- 35044
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 38637  100 38637    0     0  45189      0 --:--:-- --:--:-- --:--:-- 45242
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1708k  100 1708k    0     0   914k      0  0:00:01  0:00:01 --:--:--  914k
  % Total    % Received % Xferd  Average Speed   Tim

In [63]:
!python -m scripts.run_language_modeling \
                --train_data_file hyptrain.txt \
                --line_by_line \
                --output_dir ../gdrive/MyDrive/tapt_hyper \
                --model_type roberta-base \
                --tokenizer_name roberta-base \
                --mlm \
                --per_gpu_train_batch_size 8 \
                --gradient_accumulation_steps 16  \
                --model_name_or_path roberta-base \
                --eval_data_file hypdev.txt \
                --do_train \
                --num_train_epochs 100  \
                --learning_rate 0.0001 \
                --logging_steps 50

Streaming output truncated to the last 5000 lines.
Iteration:  14% 9/65 [00:03<00:19,  2.91it/s]
Iteration:  15% 10/65 [00:03<00:18,  2.92it/s]
Iteration:  17% 11/65 [00:03<00:18,  2.92it/s]
Iteration:  18% 12/65 [00:04<00:18,  2.92it/s]
Iteration:  20% 13/65 [00:04<00:17,  2.92it/s]
Iteration:  22% 14/65 [00:04<00:17,  2.93it/s]
Iteration:  23% 15/65 [00:05<00:17,  2.92it/s]
Iteration:  25% 16/65 [00:05<00:17,  2.80it/s]
Iteration:  26% 17/65 [00:05<00:16,  2.84it/s]
Iteration:  28% 18/65 [00:06<00:16,  2.86it/s]
Iteration:  29% 19/65 [00:06<00:15,  2.88it/s]
Iteration:  31% 20/65 [00:06<00:15,  2.89it/s]
Iteration:  32% 21/65 [00:07<00:15,  2.89it/s]
Iteration:  34% 22/65 [00:07<00:14,  2.90it/s]
Iteration:  35% 23/65 [00:07<00:14,  2.90it/s]
Iteration:  37% 24/65 [00:08<00:14,  2.91it/s]
Iteration:  38% 25/65 [00:08<00:13,  2.91it/s]
Iteration:  40% 26/65 [00:08<00:13,  2.91it/s]
Iteration:  42% 27/65 [00:09<00:13,  2.92it/s]
Iteration:  43% 28/65 [00:09<00:12,  2.92it/s]
Iteration:

In [66]:
!python -m scripts.run_language_modeling \
                --train_data_file cittrain.txt \
                --line_by_line \
                --output_dir ../gdrive/MyDrive/tapt_citation \
                --model_type roberta-base \
                --tokenizer_name roberta-base \
                --mlm \
                --per_gpu_train_batch_size 8 \
                --gradient_accumulation_steps 16  \
                --model_name_or_path roberta-base \
                --eval_data_file citdev.txt \
                --do_train \
                --num_train_epochs 100  \
                --learning_rate 0.0001 \
                --logging_steps 50

Streaming output truncated to the last 5000 lines.
Iteration:  55% 116/211 [00:08<00:06, 13.81it/s]
Iteration:  56% 118/211 [00:09<00:06, 14.05it/s]
Iteration:  57% 120/211 [00:09<00:06, 13.39it/s]
Iteration:  58% 122/211 [00:09<00:06, 14.07it/s]
Iteration:  59% 124/211 [00:09<00:06, 13.93it/s]
Iteration:  60% 126/211 [00:09<00:05, 14.54it/s]
Iteration:  61% 128/211 [00:09<00:06, 13.58it/s]
Iteration:  62% 130/211 [00:09<00:06, 12.89it/s]
Iteration:  63% 132/211 [00:10<00:06, 12.02it/s]
Iteration:  64% 134/211 [00:10<00:06, 12.53it/s]
Iteration:  64% 136/211 [00:10<00:05, 13.26it/s]
Iteration:  65% 138/211 [00:10<00:05, 12.67it/s]
Iteration:  66% 140/211 [00:10<00:05, 13.19it/s]
Iteration:  67% 142/211 [00:10<00:05, 13.38it/s]
Iteration:  68% 144/211 [00:10<00:05, 13.27it/s]
Iteration:  69% 146/211 [00:11<00:04, 14.18it/s]
Iteration:  70% 148/211 [00:11<00:04, 13.65it/s]
Iteration:  71% 150/211 [00:11<00:04, 14.08it/s]
Iteration:  72% 152/211 [00:11<00:04, 14.19it/s]
Iteration:  73% 15

In [65]:
!python -m scripts.run_language_modeling \
                --train_data_file sciietrain.txt \
                --line_by_line \
                --output_dir ../gdrive/MyDrive/tapt_sciie \
                --model_type roberta-base \
                --tokenizer_name roberta-base \
                --mlm \
                --per_gpu_train_batch_size 8 \
                --gradient_accumulation_steps 16  \
                --model_name_or_path roberta-base \
                --eval_data_file sciiedev.txt \
                --do_train \
                --num_train_epochs 100  \
                --learning_rate 0.0001 \
                --logging_steps 50

Streaming output truncated to the last 5000 lines.
Iteration:  67% 272/403 [00:16<00:08, 15.78it/s]
Iteration:  68% 274/403 [00:16<00:07, 16.79it/s]
Iteration:  69% 277/403 [00:16<00:07, 17.53it/s]
Iteration:  69% 279/403 [00:16<00:07, 17.66it/s]
Iteration:  70% 281/403 [00:16<00:07, 17.20it/s]
Iteration:  70% 283/403 [00:16<00:06, 17.80it/s]
Iteration:  71% 285/403 [00:16<00:06, 18.17it/s]
Iteration:  71% 287/403 [00:17<00:06, 18.20it/s]
Iteration:  72% 289/403 [00:17<00:07, 16.03it/s]
Iteration:  72% 292/403 [00:17<00:06, 16.98it/s]
Iteration:  73% 294/403 [00:17<00:06, 16.68it/s]
Iteration:  73% 296/403 [00:17<00:06, 16.82it/s]
Iteration:  74% 298/403 [00:17<00:06, 17.40it/s]
Iteration:  74% 300/403 [00:17<00:06, 17.15it/s]
Iteration:  75% 302/403 [00:17<00:05, 17.61it/s]
Iteration:  75% 304/403 [00:18<00:06, 15.35it/s]
Iteration:  76% 306/403 [00:18<00:05, 16.23it/s]
Iteration:  77% 309/403 [00:18<00:05, 17.12it/s]
Iteration:  77% 311/403 [00:18<00:05, 16.87it/s]
Iteration:  78% 31